In [2]:
import pandas as pd
import spacy
from sklearn import model_selection
import numpy as np


from tensorflow import keras
from tensorflow.keras import layers, models

In [3]:
df = pd.read_json(open("../../data/data.json", "r", encoding="utf8"))

In [4]:
df

,title,author,year,text,themes
0,"[45, 4, 141, 903]",0,2002,"[2, 2334, 1097, 5, 2334, 298, 9780, 1002, 412,...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,[2828],1,2002,"[1, 95, 106, 4385, 3, 144, 291, 33, 1426, 8803...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,[209],2,2002,"[106, 20, 719, 21, 1474, 329, 240, 159, 421, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[1210, 40, 114, 1989, 18, 2321]",3,2002,"[329, 303, 17, 70, 8, 1, 8, 1, 43, 1345, 1, 20...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[259, 1210]",4,2002,"[9, 649, 76, 140, 544, 206, 679, 179, 107, 44,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...
17067,"[151, 18, 2, 5178]",168,1990,"[1, 395, 2499, 331, 545, 8944, 28, 2573, 93, 8...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, ..."
17068,[],1172,1990,"[2, 597, 152, 153, 200, 277, 167, 4114, 3080, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
17070,"[15, 114]",3191,1990,"[114, 17, 70, 125, 628, 2, 797, 1, 99, 1133, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
17071,"[15, 25, 1, 13]",3434,2005,"[15, 1, 13, 37, 36, 15, 1, 311, 174, 544, 38, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [5]:
# one hot encoding for authors
def encode_authors(author_code):
    qty = df.author.max()
    result = [0] * (qty + 1)
    result[author_code] = 1
    return result

df.author = df.author.apply(encode_authors)

In [6]:
max_features = 10000  # maximum number of words in vocabulari 5000
max_len = 150  # max length of string

In [18]:
df['joined_text'] = df['text'] + df['title']
df['X2'] = df['themes'] + df['author'] 
train_df, test_df = model_selection.train_test_split(df, test_size=0.1, random_state=42)
X1_train = keras.preprocessing.sequence.pad_sequences(train_df['joined_text'].to_list(), maxlen=max_len, padding='post')
X2_train = np.stack(train_df['X2'])
Y_train = np.stack(train_df['year'])
X1_test = keras.preprocessing.sequence.pad_sequences(list(test_df['joined_text']), maxlen=max_len, padding='post')
X2_test = np.stack(test_df['X2'])
Y_test = np.stack(test_df['year'])

### Model 

In [19]:
embedding_dim =64
model1 = keras.models.Sequential([
  keras.layers.Embedding(input_dim=max_features,
                           output_dim=embedding_dim,
                           input_length=max_len),
  keras.layers.Flatten(),
  keras.layers.Dense(2000,activation='relu'),
  keras.layers.Dense(500,activation='relu'),
  keras.layers.Dense(100,activation='relu'),
  keras.layers.Dense(1, activation='relu')
])

model1.compile(optimizer='nadam',
              loss='mean_squared_error',
              metrics=['MAE'])

model1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 150, 64)           640000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 9600)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 2000)              19202000  
_________________________________________________________________
dense_9 (Dense)              (None, 500)               1000500   
_________________________________________________________________
dense_10 (Dense)             (None, 100)               50100     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 101       
Total params: 20,892,701
Trainable params: 20,892,701
Non-trainable params: 0
__________________________________________

In [20]:
epochs = 5
model1.fit(X1_train, Y_train,
          #batch_size=128,
          validation_data=(X1_test,Y_test),
          epochs=epochs)

Epoch 1/5
385/385 [==============================] - 88s 229ms/step - loss: 141485.2812 - MAE: 189.7935 - val_loss: 10381.7842 - val_MAE: 82.9884
Epoch 2/5
385/385 [==============================] - 89s 230ms/step - loss: 16184.9746 - MAE: 98.7228 - val_loss: 9193.0645 - val_MAE: 76.8472
Epoch 3/5
385/385 [==============================] - 87s 227ms/step - loss: 12304.0918 - MAE: 87.8769 - val_loss: 5372.7788 - val_MAE: 52.9201
Epoch 4/5
385/385 [==============================] - 88s 229ms/step - loss: 10282.0518 - MAE: 80.5133 - val_loss: 5044.4580 - val_MAE: 51.7279
Epoch 5/5
385/385 [==============================] - 88s 228ms/step - loss: 9399.6963 - MAE: 76.6931 - val_loss: 5334.0562 - val_MAE: 51.7882


In [21]:
score1 = model1.evaluate(np.array(X1_test), np.array(Y_test)) 

print("Test Score:", score1[0])
print("Test Accuracy:", score1[1])

43/43 [==============================] - 1s 14ms/step - loss: 5334.0562 - MAE: 51.7882
Test Score: 5334.05615234375
Test Accuracy: 51.788211822509766


In [12]:
'''
model2 = keras.models.Sequential([
  keras.layers.Dense(units=2000, activation='relu', input_shape=[len(X2_train[0])]),
  
  keras.layers.Dense(2000,activation='relu'),
  keras.layers.Dense(500,activation='relu'),
  keras.layers.Dense(100,activation='relu'),
  keras.layers.Dense(1, activation='relu')
])

model2.build()'''

In [8]:
model2 = keras.models.Sequential([
  keras.layers.Embedding(input_dim=2,
                           output_dim=8,
                           input_length=len(X2_train[0])),
  keras.layers.Flatten(),
  keras.layers.Dense(2000,activation='relu'),
  keras.layers.Dense(500,activation='relu'),
  keras.layers.Dense(100,activation='relu'),
  keras.layers.Dense(1, activation='relu')
])

model2.build()

In [9]:
model2.compile(optimizer='nadam',
              loss='mean_squared_error',
              metrics=['MAE'])

In [10]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 4116, 8)           16        
_________________________________________________________________
flatten (Flatten)            (None, 32928)             0         
_________________________________________________________________
dense (Dense)                (None, 2000)              65858000  
_________________________________________________________________
dense_1 (Dense)              (None, 500)               1000500   
_________________________________________________________________
dense_2 (Dense)              (None, 100)               50100     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 66,908,717
Trainable params: 66,908,717
Non-trainable params: 0
____________________________________________

In [11]:
model2.fit(X2_train, Y_train, epochs=5, validation_split=0.1)

Epoch 1/5
347/347 [==============================] - 277s 797ms/step - loss: 90519.9766 - MAE: 137.9661 - val_loss: 36406.3164 - val_MAE: 187.6237
Epoch 2/5
347/347 [==============================] - 240s 693ms/step - loss: 14187.0254 - MAE: 95.0268 - val_loss: 7031.6729 - val_MAE: 66.9134
Epoch 3/5
347/347 [==============================] - 247s 711ms/step - loss: 13675.5176 - MAE: 92.3169 - val_loss: 5841.9893 - val_MAE: 58.5031
Epoch 4/5
347/347 [==============================] - 244s 704ms/step - loss: 9364.1357 - MAE: 78.2141 - val_loss: 2604.0647 - val_MAE: 36.5821
Epoch 5/5
347/347 [==============================] - 242s 698ms/step - loss: 7796.7041 - MAE: 67.4034 - val_loss: 2205.5088 - val_MAE: 23.5959


In [12]:
score2 = model2.evaluate(np.array(X2_test), np.array(Y_test)) 

print("Test Score:", score2[0])
print("Test Accuracy:", score2[1])

43/43 [==============================] - 2s 45ms/step - loss: 2023.9536 - MAE: 23.0112
Test Score: 2023.95361328125
Test Accuracy: 23.011194229125977


In [25]:
y_pred1 = model1.predict(X1_test)
y_pred2 = model2.predict(X2_test)
result = (y_pred1 + y_pred2)/2

In [29]:
keras.metrics.mean_absolute_error(
    y_pred1.flatten(), y_pred2.flatten()
)

<tf.Tensor: shape=(), dtype=float32, numpy=45.97911>

### Multi-input model

In [34]:
text_input = keras.Input(shape=(max_len,))
categorical_input = keras.Input(shape=(len(X2_train[0]),))

text_embedding = layers.Embedding(max_features, 64)(text_input)
categorical_embedding = layers.Embedding(2, 8)(categorical_input)

flat_text = layers.Flatten()(text_embedding)
flat_categories = layers.Flatten()(categorical_embedding)

concatenated = keras.layers.Concatenate()([flat_text, flat_categories])

dense1 = keras.layers.Dense(2000, activation='relu', )(concatenated)
dense2 = keras.layers.Dense(500, activation='relu', )(dense1)
dense3 = keras.layers.Dense(100, activation='relu', )(dense2)
out = keras.layers.Dense(1, activation='relu', )(dense3)


united_model = keras.Model(inputs=[text_input, categorical_input], outputs=out)


In [35]:
united_model.compile(optimizer='nadam',
              loss='mean_squared_error',
              metrics=['MAE'])
united_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 4116)]       0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 150, 64)      640000      input_7[0][0]                    
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 4116, 8)      16          input_8[0][0]                    
____________________________________________________________________________________________

In [36]:
united_model.fit([X1_train, X2_train], Y_train, epochs=5, validation_split=0.1)

Epoch 1/5
347/347 [==============================] - 309s 891ms/step - loss: 88809.9531 - MAE: 143.0949 - val_loss: 3243.6409 - val_MAE: 46.2809
Epoch 2/5
347/347 [==============================] - 311s 897ms/step - loss: 13986.5469 - MAE: 99.2000 - val_loss: 6614.6440 - val_MAE: 67.1657
Epoch 3/5
347/347 [==============================] - 312s 898ms/step - loss: 11291.5625 - MAE: 88.8946 - val_loss: 4938.8389 - val_MAE: 61.5934
Epoch 4/5
347/347 [==============================] - 311s 896ms/step - loss: 7745.2114 - MAE: 68.3477 - val_loss: 4401.6143 - val_MAE: 57.2549
Epoch 5/5
347/347 [==============================] - 310s 894ms/step - loss: 7208.7290 - MAE: 68.1534 - val_loss: 2147.5571 - val_MAE: 28.1049


In [39]:
score3 = united_model.evaluate([np.array(X1_test), np.array(X2_test)], np.array(Y_test)) 

print("Test Score:", score3[0])
print("Test Accuracy:", score3[1])

43/43 [==============================] - 3s 59ms/step - loss: 1949.2349 - MAE: 27.5944
Test Score: 1949.23486328125
Test Accuracy: 27.59442901611328


In [20]:
output_dim = 100

inputs = keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, output_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

predictions = layers.Dense(1, activation='relu', name="predictions")(x)

model3 = keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model3.compile(loss="mean_squared_error", optimizer="adam", metrics=["MAE"])


In [21]:
model3.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 100)         1000000   
_________________________________________________________________
dropout (Dropout)            (None, None, 100)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 128)         89728     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         114816    
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               16512 

In [22]:
batch_size = 64
epochs = 5
model3.fit(X1_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Epoch 1/5
174/174 [==============================] - 5s 30ms/step - loss: 842177.5000 - MAE: 635.3064 - val_loss: 25712.9609 - val_MAE: 116.9544
Epoch 2/5
174/174 [==============================] - 5s 29ms/step - loss: 85895.0781 - MAE: 234.0970 - val_loss: 18861.3516 - val_MAE: 104.8669
Epoch 3/5
174/174 [==============================] - 5s 29ms/step - loss: 77035.4688 - MAE: 221.5358 - val_loss: 12337.6826 - val_MAE: 80.1150
Epoch 4/5
174/174 [==============================] - 5s 29ms/step - loss: 73586.9062 - MAE: 216.4773 - val_loss: 14829.7373 - val_MAE: 94.7481
Epoch 5/5
174/174 [==============================] - 5s 29ms/step - loss: 73614.9844 - MAE: 215.3411 - val_loss: 9908.3105 - val_MAE: 68.6175


In [23]:
accr = model3.evaluate(X1_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

43/43 [==============================] - 0s 4ms/step - loss: 7898.9966 - MAE: 63.7953
Test set
  Loss: 7898.997
  Accuracy: 63.795
